<a href="https://colab.research.google.com/github/bbhatt001/Transformer-HuggingFace-examples/blob/main/Topic_model_cnn_dailymail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imp links
# https://github.com/huggingface/transformers/blob/master/examples/pytorch/summarization/README.md
# https://colab.research.google.com/github/huggingface/datasets/blob/master/notebooks/Overview.ipynb


In [2]:
! pip install datasets transformers

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [3]:
import transformers

print(transformers.__version__)

4.12.5


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

## Preparing the dataset

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [5]:
from datasets import load_dataset, load_metric

In [6]:
from pprint import pprint

In [7]:
#version number is needed for cnn-dailymail dataset; version 3 used
# details here-- https://huggingface.co/datasets/cnn_dailymail
#'summarization': Versions 2.0.0 and 3.0.0 of the CNN / DailyMail Dataset can be used to train a model for abstractive and extractive summarization (Version 1.0.0 was developed for machine reading and comprehension and abstractive question answering). The model performance is measured by how high the output summary's ROUGE score for a given article is when compared to the highlight as written by the original article author. Zhong et al (2020) report a ROUGE-1 score of 44.41 when testing a model trained for extractive summarization. See the Papers With Code leaderboard for more models.

In [8]:
dataset = load_dataset('cnn_dailymail', '3.0.0')

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
# details about the dataset ; it is organised as a dictionary
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [10]:
# dataset made up of a dictionary
doc_train_0 = dataset['train'][0]
doc_train_0

{'article': 'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but bec

In [11]:
doc_train_0_article = dataset['train'][0]['article']
doc_train_0_article

'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he want

In [12]:
!pip install --user -U nltk

In [13]:
import nltk
from nltk.corpus import stopwords

In [14]:
!pip install -U spacy

In [15]:
import spacy

In [16]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 4.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [17]:
nlp = spacy.load("en_core_web_sm")

In [18]:
doc_train_article = dataset['train']['article']

In [19]:
doc_train_article[:5]

['It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he wan

In [20]:
doc_validation_article = dataset['validation']['article']

In [23]:
doc_lst = [[doc] for doc in doc_validation_article]

In [24]:
doc_lst[:5]

[['(CNN)Singer-songwriter David Crosby hit a jogger with his car Sunday evening, a spokesman said. The accident happened in Santa Ynez, California, near where Crosby lives. Crosby was driving at approximately 50 mph when he struck the jogger, according to California Highway Patrol Spokesman Don Clotworthy. The posted speed limit was 55. The jogger suffered multiple fractures, and was airlifted to a hospital in Santa Barbara, Clotworthy said. His injuries are not believed to be life threatening. "Mr. Crosby was cooperative with authorities and he was not impaired or intoxicated in any way. Mr. Crosby did not see the jogger because of the sun," said Clotworthy. According to the spokesman, the jogger and Crosby were on the same side of the road. Pedestrians are supposed to be on the left side of the road walking toward traffic, Clotworthy said. Joggers are considered pedestrians. Crosby is known for weaving multilayered harmonies over sweet melodies. He belongs to the celebrated rock grou

In [25]:
docs_nlp = [nlp(doc) for doc in doc_validation_article]

In [26]:
docs_nlp[:5]

[(CNN)Singer-songwriter David Crosby hit a jogger with his car Sunday evening, a spokesman said. The accident happened in Santa Ynez, California, near where Crosby lives. Crosby was driving at approximately 50 mph when he struck the jogger, according to California Highway Patrol Spokesman Don Clotworthy. The posted speed limit was 55. The jogger suffered multiple fractures, and was airlifted to a hospital in Santa Barbara, Clotworthy said. His injuries are not believed to be life threatening. "Mr. Crosby was cooperative with authorities and he was not impaired or intoxicated in any way. Mr. Crosby did not see the jogger because of the sun," said Clotworthy. According to the spokesman, the jogger and Crosby were on the same side of the road. Pedestrians are supposed to be on the left side of the road walking toward traffic, Clotworthy said. Joggers are considered pedestrians. Crosby is known for weaving multilayered harmonies over sweet melodies. He belongs to the celebrated rock group 

In [27]:
docs_nlp[0].ents[0].lemma_

'cnn)singer-songwriter David Crosby'

In [28]:
allowed_pos = ['NOUN', 'ADJ', "PROPN", "VERB"]

In [29]:
lemmas = [[token.lemma_ for token in doc_nlp if token.pos_ in allowed_pos and not token.is_stop] for doc_nlp in docs_nlp]

In [30]:
import itertools

In [31]:
import pandas as pd

In [32]:
from collections import Counter

In [33]:
terms =  list(itertools.chain.from_iterable(lemmas))
count_terms = Counter(terms)
pd_terms = pd.Series(count_terms)
sorted_count = pd_terms.sort_values(ascending =False)
sorted_count


say          59208
year         32283
time         18257
tell         14553
take         13649
             ...  
Xishan           1
Pollution        1
biomass          1
Guannan          1
toilete          1
Length: 97001, dtype: int64

In [34]:
lemmas[:3]

[['cnn)singer',
  'songwriter',
  'David',
  'Crosby',
  'hit',
  'jogger',
  'car',
  'Sunday',
  'evening',
  'spokesman',
  'say',
  'accident',
  'happen',
  'Santa',
  'Ynez',
  'California',
  'Crosby',
  'live',
  'Crosby',
  'drive',
  'mph',
  'strike',
  'jogger',
  'accord',
  'California',
  'Highway',
  'Patrol',
  'Spokesman',
  'Don',
  'Clotworthy',
  'post',
  'speed',
  'limit',
  'jogger',
  'suffer',
  'multiple',
  'fracture',
  'airlift',
  'hospital',
  'Santa',
  'Barbara',
  'Clotworthy',
  'say',
  'injury',
  'believe',
  'life',
  'threaten',
  'Mr.',
  'Crosby',
  'cooperative',
  'authority',
  'impaired',
  'intoxicate',
  'way',
  'Mr.',
  'Crosby',
  'jogger',
  'sun',
  'say',
  'Clotworthy',
  'accord',
  'spokesman',
  'jogger',
  'Crosby',
  'road',
  'pedestrian',
  'suppose',
  'left',
  'road',
  'walk',
  'traffic',
  'Clotworthy',
  'say',
  'jogger',
  'consider',
  'pedestrian',
  'Crosby',
  'know',
  'weave',
  'multilayere',
  'harmony',
 

In [40]:
#lemmas1 = [[token.lemma_ for token in doc_nlp if token.pos_ in allowed_pos and not token.is_stop] for doc_nlp in docs_nlp[:5]]

In [38]:
#lemmas2 = [[token.lemma_ for token in doc_nlp if token.pos_ in allowed_pos] for doc_nlp in docs_nlp[:5]]

In [21]:
!pip install gensim 

In [22]:
!pip install pyLDAvis

In [49]:
import pyLDAvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [43]:
import gensim

In [35]:
from gensim.test.utils import datapath

In [38]:
# create a dictionary
from gensim.corpora import Dictionary

In [39]:
dictionary1 = Dictionary(lemmas)

In [40]:
# convert dictionary into bag-of-words (BoW)/document-term-matrix
corpus = [dictionary1.doc2bow(text) for text in lemmas]

In [41]:
len(dictionary1)

97001

In [44]:
# instatiating an lda model
LDA = gensim.models.ldamodel.LdaModel 

In [45]:
lda_model_cnn = LDA(corpus = corpus, id2word = dictionary1, num_topics =50, random_state =1)

In [47]:
lda_model_cnn.print_topics(num_topics =50, num_words =20)

[(0,
  '0.029*"United" + 0.013*"Manchester" + 0.010*"Gaal" + 0.010*"Arsenal" + 0.008*"season" + 0.008*"Old" + 0.008*"League" + 0.008*"Van" + 0.007*"Trafford" + 0.007*"van" + 0.007*"year" + 0.006*"club" + 0.006*"say" + 0.006*"look" + 0.006*"Louis" + 0.005*"Premier" + 0.005*"time" + 0.005*"Maria" + 0.004*"win" + 0.004*"play"'),
 (1,
  '0.009*"say" + 0.007*"water" + 0.006*"Earth" + 0.006*"surface" + 0.005*"image" + 0.005*"year" + 0.005*"light" + 0.005*"scientist" + 0.005*"air" + 0.005*"eclipse" + 0.004*"mile" + 0.004*"system" + 0.004*"show" + 0.004*"space" + 0.004*"sun" + 0.004*"solar" + 0.004*"create" + 0.004*"fly" + 0.004*"China" + 0.004*"large"'),
 (2,
  '0.016*"food" + 0.015*"say" + 0.013*"child" + 0.009*"health" + 0.007*"people" + 0.006*"year" + 0.006*"cent" + 0.005*"eat" + 0.005*"study" + 0.005*"day" + 0.005*"school" + 0.005*"find" + 0.005*"time" + 0.004*"help" + 0.004*"age" + 0.004*"good" + 0.004*"brain" + 0.004*"start" + 0.004*"research" + 0.004*"baby"'),
 (3,
  '0.020*"say" + 0.0

In [54]:
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [57]:
cnn_topic50_vis = gensimvis.prepare(lda_model_cnn, corpus, dictionary1)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [58]:
cnn_topic50_vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
31     0.239703  0.091140       1        1  8.800188
44    -0.109617  0.085384       2        1  8.125600
41    -0.001949 -0.002004       3        1  6.143754
4     -0.023027  0.033886       4        1  3.794814
37     0.189244  0.066214       5        1  3.677222
22    -0.053100  0.047179       6        1  3.098904
10    -0.067970  0.078922       7        1  3.033200
25    -0.022735  0.035657       8        1  2.909115
36     0.177435  0.067811       9        1  2.881024
8     -0.136463  0.116410      10        1  2.685302
1     -0.034714 -0.169794      11        1  2.536490
38     0.066622 -0.138507      12        1  2.449505
18     0.026674 -0.089532      13        1  2.387020
28    -0.041533 -0.053233      14        1  2.365482
14    -0.125861  0.091443      15        1  2.345969
2     -0.008724 -0.091864      16        1  2.201804
26    -0.015092 -0.146648      17        1  2.051647
12    -0.036763 -0.047123      18        1  2.025572
39    -0.065652 -0.066152      19        1  1.955403
11    -0.076832 -0.045488      20        1  1.946379
17    -0.056713  0.070791      21        1  1.881002
0      0.179836  0.040447      22        1  1.789997
29     0.111753  0.025569      23        1  1.754653
49     0.141075 -0.011615      24        1  1.601145
13     0.022676  0.025908      25        1  1.557219
16     0.072638 -0.135418      26        1  1.498683
3     -0.068231  0.014052      27        1  1.479773
9     -0.078887  0.061955      28        1  1.418399
24    -0.076867  0.064815      29        1  1.357012
7     -0.010239 -0.085688      30        1  1.258715
35     0.087949  0.023696      31        1  1.239645
20     0.151639  0.064212      32        1  1.196663
40    -0.041403 -0.076481      33        1  1.185947
33    -0.035714 -0.104355      34        1  1.019237
43    -0.075624  0.030333      35        1  1.004748
5     -0.088413  0.033561      36        1  0.976144
15    -0.036956  0.079116      37        1  0.960579
19    -0.002048 -0.022599      38        1  0.959249
45    -0.042685  0.045806      39        1  0.919726
6     -0.051845 -0.000598      40        1  0.918741
30    -0.021706  0.064963      41        1  0.913283
21     0.100645  0.006845      42        1  0.837659
23    -0.043282 -0.051015      43        1  0.765612
32     0.022856 -0.003952      44        1  0.734276
47     0.092647 -0.054441      45        1  0.646204
46    -0.020494  0.048440      46        1  0.640052
48    -0.030162 -0.047576      47        1  0.583128
34    -0.023858 -0.044889      48        1  0.538018
42    -0.094629  0.028827      49        1  0.506671
27     0.036399  0.045592      50        1  0.443425, topic_info=         Term          Freq         Total Category  logprob  loglift
59        say  57458.000000  57458.000000  Default  30.0000  30.0000
1100   school   4887.000000   4887.000000  Default  29.0000  29.0000
8856       Mr  11695.000000  11695.000000  Default  28.0000  28.0000
596    police   8325.000000   8325.000000  Default  27.0000  27.0000
195   student   3081.000000   3081.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
199      time     76.279343  18006.555344  Topic50  -5.5348  -0.0457
209     world     62.309972   5418.081932  Topic50  -5.7370   0.9530
166     month     61.892838   8218.858284  Topic50  -5.7438   0.5296
210     write     60.404389   4244.409929  Topic50  -5.7681   1.1661
2900    visit     57.317283   3064.098337  Topic50  -5.8206   1.4395

[4070 rows x 6 columns], token_table=       Topic      Freq Term
term                       
4160      25  0.912711    !
4160      28  0.040565    !
4160      33  0.020282    !
7650       1  0.023448    #
7650       2  0.005211    #
...      ...       ...  ...
28332     44  0.001353    £
28332     45  0.002029    £
28332     46  0.000676    £
28332     47  0.001353    £

In [64]:
pyLDAvis.save_html(cnn_topic50_vis, 'cnn_topic50vis.html')

In [65]:
from google.colab import files
files.download('cnn_topic50vis.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>